In [1]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected', 'stubby', 'strict_thin', 'U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                elif group.name.startswith("Stubby"):
                    spine_type = 'stubby'
                elif group.name.startswith("Strict"):
                    spine_type = 'strict_thin'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

[2024-01-01 10:11:52,762][INFO]: Connecting MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306
[2024-01-01 10:11:53,189][INFO]: Connected MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306


Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+

 (Total: 0)



In [28]:
#drop all the tables
mouse.drop()

[2023-09-08 12:06:56,734][INFO]: `MarikeReimer`.`mouse` (4 tuples)
[2023-09-08 12:06:56,862][INFO]: `MarikeReimer`.`session` (19 tuples)
[2023-09-08 12:06:57,019][INFO]: `MarikeReimer`.`dendrite` (19 tuples)
[2023-09-08 12:06:57,161][INFO]: `MarikeReimer`.`_image_segmentation` (575 tuples)
[2023-09-08 12:06:57,302][INFO]: `MarikeReimer`.`__distance_to_soma` (575 tuples)
[2023-09-08 12:06:59,675][INFO]: Dropped table `MarikeReimer`.`__distance_to_soma`
[2023-09-08 12:07:00,005][INFO]: Dropped table `MarikeReimer`.`_image_segmentation`
[2023-09-08 12:07:00,349][INFO]: Dropped table `MarikeReimer`.`dendrite`
[2023-09-08 12:07:00,643][INFO]: Dropped table `MarikeReimer`.`session`
[2023-09-08 12:07:00,949][INFO]: Dropped table `MarikeReimer`.`mouse`
[2023-09-08 12:07:01,160][INFO]: Tables dropped. Restart kernel.


In [2]:
#For general load checking
mouse*session

subject_id Primary keys above the '---',identifier Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology


In [45]:
#Check if a specific subject loaded
session & 'subject_id = "L882"'

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L882,Dendrite1,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite2a,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite2b,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite3b,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite4a,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite4b,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite4c,"Contusion: 50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite5,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite5F2,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI
L882,Dendrite5F3,"Contusion: (50 kDyn) SCI at the thoracic vertebral level 11 (T11), lumbar segment 2 (L2)",Vehicle 1% DMSO inject IP 4 weeks after SCI


In [46]:
#Delete a Subject with an identifier - use after failing QA on a Blender file
(session & 'subject_id = "L882"' & 'identifier = "Dendrite4c"').delete()

[2023-09-08 12:39:30,438][INFO]: Deleting 24 rows from `MarikeReimer`.`__distance_to_soma`
[2023-09-08 12:39:30,718][INFO]: Deleting 24 rows from `MarikeReimer`.`_image_segmentation`
[2023-09-08 12:39:31,012][INFO]: Deleting 1 rows from `MarikeReimer`.`dendrite`
[2023-09-08 12:39:31,306][INFO]: Deleting 1 rows from `MarikeReimer`.`session`
[2023-09-08 12:39:33,108][INFO]: Deletes committed.


1

In [15]:
#Display info for a Subject with an identifier 

(session & image_segmentation & distance_to_soma & 'subject_id = "L831"' & 'identifier = "Dendrite1"')

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology


In [5]:
#Display info for a Subject with an identifier 

frame = pd.DataFrame((mouse&'subject_id = "L831"')*(session & 'identifier = "Dendrite3"')*image_segmentation)
#(session & image_segmentation & 'subject_id = "L736"' & 'identifier = "Dendrite1"'& distance_to_soma )
frame

11

In [3]:
#Pivot tables in Excel exclude blank values, so I download this separately to make sure spineless dendrites don't get missed
frame = pd.DataFrame(mouse*session*dendrite)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [30]:
#Export pipeline to csv
frame = pd.DataFrame(mouse*session*dendrite*image_segmentation*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/UnfilteredDiscData_Vsad_dendritestart.csv")
frame

,subject_id,identifier,dendrite_id,segmentation_name,genotype,sex,species,strain,surgery,pharmacology,soma_center_point,proximal_dendrite_length,medial_dendrite_length,distal_dendrite_length,length,volume,surface_area,spine_type,center_of_mass,distance_to_soma
0,L691,Dendrite1a,37,MushroomSpine_1_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[42.5914, 106.28, 8.1472]",28.6961,28.1198,0.0,0.653247,0.057056,0.901271,mushroom,"[69.87845611572266, 88.1337661743164, 7.309266...",32.78070
1,L691,Dendrite1a,37,StubbySpine_17_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[42.5914, 106.28, 8.1472]",28.6961,28.1198,0.0,0.504146,0.214775,1.606340,stubby,"[48.11497116088867, 104.91754150390625, 5.5403...",6.25795
2,L691,Dendrite1a,37,StubbySpine_18_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[42.5914, 106.28, 8.1472]",28.6961,28.1198,0.0,0.370939,0.290909,2.091710,stubby,"[56.04249954223633, 102.64118194580078, 6.9680...",13.98440
3,L691,Dendrite1a,37,StubbySpine_19_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[42.5914, 106.28, 8.1472]",28.6961,28.1198,0.0,0.367342,0.167758,1.397160,stubby,"[67.04827880859375, 93.90428161621094, 7.32299...",27.42220
4,L691,Dendrite1a,37,StubbySpine_20_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[42.5914, 106.28, 8.1472]",28.6961,28.1198,0.0,0.173103,0.007566,0.196070,stubby,"[66.54460144042969, 90.60702514648438, 7.27345...",28.63850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,L902,Dendrite2,30,ThinSpine_3_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion: (50 kDyn) SCI at the thoracic verte...,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[47.1559, 42.5123, 0.449925]",28.7797,30.6071,0.0,0.618253,0.058634,0.805853,thin,"[45.94371795654297, 53.20823287963867, 5.04025...",11.70230
872,L902,Dendrite2,30,ThinSpine_4_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion: (50 kDyn) SCI at the thoracic verte...,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[47.1559, 42.5123, 0.449925]",28.7797,30.6071,0.0,0.595454,0.019660,0.480792,thin,"[58.30590057373047, 53.45787048339844, 1.17546...",15.64140
873,L902,Dendrite2,30,ThinSpine_5_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion: (50 kDyn) SCI at the thoracic verte...,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[47.1559, 42.5123, 0.449925]",28.7797,30.6071,0.0,0.444355,0.045750,0.700220,thin,"[58.9035530090332, 53.76449203491211, 1.461903...",16.29860
874,L902,Dendrite2,30,ThinSpine_6_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Contusion: (50 kDyn) SCI at the thoracic verte...,Vehicle 1% DMSO inject IP 4 weeks after SCI,"[47.1559, 42.5123, 0.449925]",28.7797,30.6071,0.0,1.921410,0.660939,4.133450,thin,"[55.32374954223633, 70.32195281982422, 4.57632...",29.27660


In [9]:
#View pipeline
#mouse*(session& 'surgery = "Contusion"')*dendrite*image_segmentation*distance_to_soma
mouse*(session)*dendrite*image_segmentation*distance_to_soma

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology,soma_center_point,proximal_dendrite_length,medial_dendrite_length,distal_dendrite_length,length,volume,surface_area,spine_type,center_of_mass,distance_to_soma
L691,Dendrite1a,37,MushroomSpine_1_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.653247,0.0570558,0.901271,mushroom,=BLOB=,32.7807
L691,Dendrite1a,37,StrictThinSpine_6_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.961704,0.112817,1.4235,strict_thin,=BLOB=,61.3321
L691,Dendrite1a,37,StubbySpine_17_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.504146,0.214775,1.60634,stubby,=BLOB=,6.25795
L691,Dendrite1a,37,StubbySpine_18_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.370939,0.290909,2.09171,stubby,=BLOB=,13.9844
L691,Dendrite1a,37,StubbySpine_19_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.367342,0.167758,1.39716,stubby,=BLOB=,27.4222
L691,Dendrite1a,37,StubbySpine_20_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.173103,0.00756634,0.19607,stubby,=BLOB=,28.6385
L691,Dendrite1a,37,StubbySpine_21_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.19602,0.0408258,0.531692,stubby,=BLOB=,34.2983
L691,Dendrite1a,37,ThinSpine_7_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.6961,28.1198,0.0,0.260867,0.00151419,0.0896392,thin,=BLOB=,65.1893
L691,Dendrite1b,36,StrictThinSpine_1_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.3198,31.4149,0.0,1.11659,0.197682,1.80441,strict_thin,=BLOB=,71.5847
L691,Dendrite1b,36,StubbySpine_0_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,28.3198,31.4149,0.0,0.292509,0.0178378,0.196053,stubby,=BLOB=,15.5293


In [47]:
#Query for dataset 1.3+

frame = pd.DataFrame(mouse*session*dendrite*(image_segmentation & 'volume >= "0.01"' & 'volume <= "0.8"' &'length >= "0.2"' & 'length <= "3"')*distance_to_soma)
print(frame)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/DiscSpinedata_V1.5.csv")

    subject_id  identifier  dendrite_id                  segmentation_name  \
0         L691  Dendrite1a           37  MushroomSpine_1_plane_segmentaton   
1         L691  Dendrite1a           37   StubbySpine_17_plane_segmentaton   
2         L691  Dendrite1a           37   StubbySpine_18_plane_segmentaton   
3         L691  Dendrite1a           37   StubbySpine_19_plane_segmentaton   
4         L691  Dendrite1a           37      ThinSpine_6_plane_segmentaton   
..         ...         ...          ...                                ...   
640       L902   Dendrite2           30      ThinSpine_3_plane_segmentaton   
641       L902   Dendrite2           30      ThinSpine_4_plane_segmentaton   
642       L902   Dendrite2           30      ThinSpine_5_plane_segmentaton   
643       L902   Dendrite2           30      ThinSpine_6_plane_segmentaton   
644       L902   Dendrite2           30      ThinSpine_7_plane_segmentaton   

     genotype sex       species   strain  \
0    Thy1-YFP   F  

In [32]:
#Didn't meet criteria because too small

frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume <= "0.01"' or 'length < "0.2" ' )*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toosmall.csv")
print(len(frame))

91


In [33]:
#Didn't meet criteria because too big
frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume > "0.8"' or 'length > "3"')*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toobig.csv")
print(len(frame))

23


In [3]:
mouse

subject_id Primary keys above the '---',genotype,sex,species,strain
